<CENTER>
<img src="../../../../media/summer_school_banner.png"
WIDTH=900 HEIGHT=450>
</CENTER>

<CENTER>
<p><font size="7"><b>IMT Summer School 2024</b></font></p>
<p><font size="5">Dynamic resource allocation problems in communication networks</font></p>
<p><font size="5"><b>June 25-27: </b>Day 1 - <i>Resource allocation problems & Restless Bandit</i></font></p>
</CENTER>

**Usefull links:**

- Summer school [website](https://sites.google.com/view/alexandre-reiffers/courses/dynamic-resource-allocation-problems-in-communication-networks).
- Organized by the team of the [RAMONaaS project](https://sites.google.com/view/ramonaas/home) at IMT Atlantique in Brest, France.
- GitHub [repository](https://github.com/ramonaas/summer-school)
- Introduction to Numpy [here](https://github.com/brain-bzh/introduction-to-ai/blob/main/session1/lab/TP0.ipynb), and a more thorough overview [here](https://www.w3schools.com/python/numpy/default.asp).
- Solve convex problems using the [CVXPY library](https://www.cvxpy.org/).


<RIGHT>
<div style="text-align: right">
<img
src="https://www.imt-atlantique.fr/sites/default/files/Images/Ecole/charte-graphique/IMT_Atlantique_logo_RVB_Baseline_400x272.jpg"
width="100" height="68">
</div>
</RIGHT>

----------------------------
----------------------------

# Introduction

In this lab session, we will explore how to model resource allocation problems in communication networks using Markov Chains. We will also delve into the concept of Restless Bandit problems and their applications.

**Topics Covered:**
1. Markov Chains and Queuing Systems
4. Solving MDPs with Linear Programming
5. Restless Bandit Problem

**How to Cite:**

```
Lucas Lopes and Alexandre Reiffers. (2024). "RAMONaaS Summer School 2024 - Day 1: Resource Allocation Problems & Restless Bandit". Retrieved from https://github.com/ramonaas/summer-school
```

# Markov Chain
> Objective: Modeling a resource allocation problem as a Markov Chain

## Queuing system
> Model: Access control for one queue

<!-- <RIGHT>
<div style="text-align: right">
<img
src="/Users/lucas/Code/lp-learner-dev/media/images/illustrations/queue_jobs.webp"
width="100" height="100">
</div>
</RIGHT> -->

### Buffer size

A queue has a possible length of $\{0,\ldots,K\}$.

> Note: It's a finite queue with $K+1$ possible length of jobs.

### Job arrivals

At each instant $t=0,\ldots,T-1$, a job arrives to be processed by a data center.
> Note: It's a finite horizon of $T$ arrivals.

### System state

We denote $S(t)$ as the size of the queue at instant $t$.

> Note: the *state* at each *time* tracks the system evolution.

### Decision-making action

We denote $A(t)$ as the decision of the scheduler of accepting ($A(t)=1$) or not ($A(t)=0$) a job in the queue at instant $t$.

> Note: If the job is accepted then it enters in tail of the queue.

## Transition probability

### System dynamics

We assume that, between two job arrivals, the number of jobs processed follow a *Binomial distribution* with parameters:

- $n:=$ number of jobs waiting to be processed in the queue **+** the possible new accepted job.
- $q\in[0,1]:=$ probability that a job is processed.

> Note: at a given instant $t$, $n := S(t) + A(t)$

#### Probability mass function
Just to recall, the PMF is:

$$
\text{Pr}(k;n,q) = \binom{n}{k} q^{k} (1-q)^{n-k}
$$

where:
- $\binom{n}{k} = \frac{n!}{k!(n-k)!}$ is the possible number of combinations of having $k$ success in $n$ trials.
- $q^{k}$ is the probability of $k$ success where each indepent success had probability $q$.
- $(1-q)^{n-k}$ is the probability of $n-k$ failures where each indepent failure had probability $(1-q)$.

> Note: The binomial distribution is the discrete probability distribution of $k$ successes in a sequence of $n$ independent trials with probability of success $q$.

### Transitioning to a next state

We denote

$\mathbb{P}(S(t+1)=y|S(t)=x,\;A(t)=a)$

as the probiblity of the next state $S(t+1)$ be $y$, knowing that:
- the current state $S(t)$ is $x$; and
- the action taken $A(t)$ was $a$.

> Note: Because of the Markovian property, knowing the current state is enough to get the probability for the next state.

#### w.r.t Processed jobs

To calculate the transition probability for $s\in\{0,\ldots,K\}$:

$$
\mathbb{P}(S(t+1)=s+a-k|S(t)=s,\;A(t)=a):= I\{k\leq a+s\}\binom{a+s}{k}q^{k}(1-q)^{a+s-k}
$$

> Note: $k$ is the number of processed jobs.

#### w.r.t Remaining jobs

Therefore, we can derive the probability of the next have $s'$ remaining jobs:

$$
\mathbb{P}(S(t+1)=s'|S(t)=s,\;A(t)=a):= I\{s'\leq a+s\}\binom{a+s}{s'}q^{a+s-s'}(1-q)^{s'}.
$$

> Note: $s'$ is the number of jobs left in the queue on the next state.

### Function implementation

#### Instantanous MDP

In [ ]:
from scipy.stats import binom

def inst_MDP(s_,s,a,q,K):  # probability of transitioning to s_ given s,a
  p = 0
  on_buffer_limit = (s+a <= K)  # on buffer limit
  no_extra_jobs = (s_ <= s+a)  # the next state s_ can not have more jobs than the current state s+a
  if on_buffer_limit and no_extra_jobs:
    p = binom.pmf(int(s_), int(a+s), 1-q)  # probability of s_ failures in a+s trials with failure rate (1-q) where q is the probability of success
  return p # probability of transitioning to s_

#### P matrix

Let us denote $p_{ss'}^a:=\mathbb{P}(S(t+1)=s'|S(t)=s,\;A(t)=a)$.

In [ ]:
import numpy as np

def P_matrix(q=0.2, K=10):  # P[s,s_,a] = inst_MDP(s_,s,a,q,K)
  return np.array([[[inst_MDP(s_,s,a,q,K) for a in range(2)] for s in range(K+1)] for s_ in range(K+1)])

### Visualizing the P matrix

#### Plot function

In [ ]:
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def heatmap_animation(Ps, vals, colorscale='deep', xaxis_title='Current State', yaxis_title='Next State', colorbar_title='Probability', use_vals_as_ticks=False):
  # Create the frames for the animation
  frames = []
  for idx, P in enumerate(Ps):
    # Create the heatmap traces for each frame
    traces = [go.Heatmap(z=P[:,:,I], name=f'P_{I}', showscale=False, coloraxis='coloraxis') for I in range(2)]
    # Create the frame for the current frame index
    frame = go.Frame(data=traces, name=f'Frame {idx}')
    frames.append(frame)
  # Create subplots with 1 row and 2 columns
  fig = make_subplots(rows=1, cols=2, subplot_titles=['Giving the action was Rejection', 'Giving the action was Acceptance'])
  # Add the traces to the subplots
  yaxis_ticks = [str(v) for v in vals] if use_vals_as_ticks else None
  for i in range(2):
    fig.add_trace(go.Heatmap(z=frames[0].data[i].z, y=yaxis_ticks, name=f'P_{i}', showscale=False, coloraxis='coloraxis'), row=1, col=i+1)
  # Create the animation figure
  fig.update(frames=frames)
  # Update the layout with sliders
  fig.update_layout(
    sliders=[{
      'active': 0,
      'currentvalue': {'prefix': 'q = '},
      'pad': {'t': 50},
      'steps': [{'args': [[f'Frame {idx}']], 'label': f'{vals[idx]}', 'method': 'animate'} for idx in range(len(frames))]}],
    xaxis=dict(title=xaxis_title),
    yaxis=dict(title=yaxis_title),
    title=None,  # Remove the main title
    coloraxis=dict(colorbar=dict(title=colorbar_title), colorscale=colorscale))
  # Update the axis labels and title for each heatmap
  for i in range(2):
    fig.update_xaxes(title_text=xaxis_title, row=1, col=i+1, ticktext=vals) # tickvals=vals, ticktext=vals)
  fig.update_yaxes(title_text=yaxis_title, row=1, col=1)
  return fig

#### Plotting

In [ ]:
# Create the transition matrices for different q values
qs = np.round(np.linspace(0, 1, 11)[1:], 2) # list of rounded q values
Ps = [P_matrix(q) for q in qs]  # list of transition matrices for different q values
fig_p = heatmap_animation(Ps, qs, colorscale='ice')  # create the animation figure
fig_p.show()

## Cost function

### Incentives

#### Incentive for accepting jobs

We define $-\gamma$ as an instantaneous decrease on cost for incentive the scheduler to accepting a job.

> Note: If the action is reject, therefore the cost will not decrease, however, if the action is accept, the cost will decrease in function of $\gamma$

#### Incentive for processing jobs

The scheduler faces a storage cost caused by the length of the queue.
We assume that the cost has the following three property:

- Equal to $0$ when $S(t)=0$;
- Increasing in $S(t)$;
- Quadratic in $S(t)$.

> Remark: We can actually take the energy cost depending of $S(t)+A(t)$ and not just depending on $S(t)$.

### Formulating the cost

$$
C(S(t),A(t)):=S(t)^2-\gamma A(t).
$$

> Note: This is an instantaneous cost that the scheduler is facing.

### Function implementation

In [ ]:
def cost(s,a,gamma):
  return s**2 - gamma*a

#### Cost Matrix

In [ ]:
def costs_matrix(gammas, K=10):
  costs = np.zeros((K+1, len(gammas), 2))
  for s in range(K+1):
    for idx, gamma in enumerate(gammas):
      for a in range(2):
        costs[idx,s,a] = cost(s,a,gamma)
  return costs

### Visualizing the Cost

Let's use a heatmap to visualize the costs for different values of $\gamma$.

#### Plotting

In [ ]:
# Create the transition matrices for different q values
gammas = np.round(np.linspace(0, 100, 10+2)[1:], 2) # list of rounded q values
costs = costs_matrix(gammas)  # list of transition matrices for different q values
fig_cost = heatmap_animation([costs], gammas, colorscale='dense', yaxis_title='Cost decrease for acceptance', colorbar_title='Cost', use_vals_as_ticks=True)  # create the animation figure
fig_cost.show()

## Parameters setting

Let's set the parameters for our system that will be used in the rest of the notebook.

In [ ]:
K = 10
T = 50
q = 5e-2
gamma = 1
budget = 0.5

## Trajectory simulation

### Next state simulation

We can use the following algorithm to simulate the next state $ \hat{S}(t+1) $ such that the probability $ \mathbb{P}(\hat{S}(t+1)= s' \mid S(t)=s,\;A(t)=a) $ equals the given transition probability $ p_{ss'}^a = \mathbb{P}(S(t+1) = s' \mid S(t) = s, A(t) = a)$:

1. Generate a uniform random variable $ U \sim \text{Unif}([0, 1]) $.
2. Define the next state $ \hat{S}(t+1) $ as $ s' $ if $ \sum_{s''=0}^{s'-1} p_{ss''}^a \leq U < \sum_{s''=0}^{s'} p_{ss''}^a $.

> Note: the actual transition probabilities $p_{ss'}^a$ determines the partition of the interval $[0,1]$ in $K+1$ regions, one for each possible next state. The corresponding region where $U$ falls will map to each next state the simulation will go.

#### Proof

Let's proof that $ \mathbb{P}(\hat{S}(t+1) = s' \mid S(t) = s, A(t) = a) = p_{ss'}^a $

1. **Partitioning the Interval**:
   The algorithm partitions the interval $[0, 1]$ based on the cumulative probabilities.
   
   Define the cumulative probability $ P_{ss'}^a $ as:
   $ P_{ss'}^a = \sum_{s''=0}^{s'} p_{ss''}^a $

   > Note: $ P_{ss'}^a$ is the cumulative sum of the transition probabilities from state $ s $ to all states up to $ s' $.

2. **Defining Intervals**:
   The condition in the algorithm, $ \sum_{s''=0}^{s'-1} p_{ss''}^a \leq U < \sum_{s''=0}^{s'} p_{ss''}^a $, translates to:
   
   
   $ P_{ss'-1}^a \leq U < P_{ss'}^a $
   where $ P_{ss'-1}^a = \sum_{s''=0}^{s'-1} p_{ss''}^a $.

   > Note: This interval is defined for each state $ s' $, ensuring that the intervals for different states are mutually exclusive and cover the entire interval $[0, 1]$.

3. **Probability of Next State**:
   Since $ U $ is uniformly distributed over $[0, 1]$, the probability that $ U $ falls within the interval $[P_{ss'-1}^a, P_{ss'}^a)$ is given by the length of this interval:
   
   $ \mathbb{P}(P_{ss'-1}^a \leq U < P_{ss'}^a) = P_{ss'}^a - P_{ss'-1}^a $

   > Note: the value of $U$ is between the cumulative probabilities of $\sum_{s''=0}^{s'-1} p_{ss''}^a$ and $\sum_{s''=0}^{s'} p_{ss''}^a$.

4. **Interval Length**:
   The length of the interval $[P_{ss'-1}^a, P_{ss'}^a)$ is:
   
   $ P_{ss'}^a - P_{ss'-1}^a = \left(\sum_{s''=0}^{s'} p_{ss''}^a\right) - \left(\sum_{s''=0}^{s'-1} p_{ss''}^a\right) = p_{ss'}^a $

   > Note: $P_{ss'}^a = P_{ss'-1}^a + p_{ss'}^a$.

5. **Conclusion**:
   Therefore, the probability that $ \hat{S}(t+1) = s' $ given $ S(t) = s $ and $ A(t) = a $ is:

   $ \mathbb{P}(\hat{S}(t+1) = s' \mid S(t) = s, A(t) = a) = \mathbb{P}(P_{ss'-1}^a \leq U < P_{ss'}^a) = p_{ss'}^a $

   > Note: the algorithm correctly simulates the next state $ \hat{S}(t+1) $ with the desired transition probabilities $ p_{ss'}^a $.

#### Function implementation

In [ ]:
def simulate_next_state(s,a,q,K):
  pxa = [inst_MDP(s_,s,a,q,K) for s_ in range(K+1)]  # Generate the actual transition probabilities for each state
  U = np.random.uniform(0,1,1)  # Sampling
  # Finding the next state
  S_next = 0
  cumulative = sum(pxa[:1])
  while cumulative < U and S_next < K:
    S_next += 1
    cumulative += pxa[S_next]
  return S_next  # Return the next state

#### Visualizing the interval map

##### Plot function

In [ ]:
def barplot_animation(Ps, qs, color_palette='Prism', xaxis_title='Transition Probability', yaxis_title='Current State'):
    color_palette = getattr(plotly.colors.qualitative, color_palette)
    # Create frames for the animation
    frames = []
    for idx, P in enumerate(Ps):
        traces = []
        for a in range(2):  # For each action (0 and 1)
            xs, ys, zs, colors = [], [], [], []
            for s in range(P.shape[0]):
                for s_ in range(P.shape[1]):
                    xs.append(P[:, :, a][s_, s])
                    ys.append(s)
                    zs.append(f'State {s_}')
                    colors.append(color_palette[s_ % len(color_palette)])
            traces.append(go.Bar(x=xs, y=ys, orientation='h', marker=dict(color=colors), text=zs, showlegend=False))
        frames.append(go.Frame(data=traces, name=f'Frame {idx}'))
    # Create subplots with 1 row and 2 columns
    fig = make_subplots(rows=1, cols=2, subplot_titles=['Giving the action was Rejection', 'Giving the action was Acceptance'])
    # Add initial traces to the subplots
    for a in range(2):
        xs, ys, zs, colors = [], [], [], []
        for s in range(Ps[0].shape[0]):
            for s_ in range(Ps[0].shape[1]):
                xs.append(Ps[0][:, :, a][s_, s])
                ys.append(s)
                zs.append(f'State {s_}')
                colors.append(color_palette[s_ % len(color_palette)])
        fig.add_trace(go.Bar(x=xs, y=ys, orientation='h', marker=dict(color=colors), text=zs, showlegend=False), row=1, col=a+1)
    fig.update(frames=frames) # Update frames
    # Update layout with sliders and axis titles
    fig.update_layout(
        sliders=[{
            'active': 0,
            'currentvalue': {'prefix': 'q = '},
            'pad': {'t': 50},
            'steps': [{'args': [[f'Frame {idx}']], 'label': f'{qs[idx]}', 'method': 'animate'} for idx in range(len(frames))]}],
        xaxis=dict(title=xaxis_title),
        yaxis=dict(title=yaxis_title),)
    # Update the axis labels and range for each subplot
    for a in range(2):
        fig.update_xaxes(title_text=xaxis_title, range=[0, 1], row=1, col=a+1)
    fig.update_yaxes(title_text=yaxis_title, row=1, col=1)
    return fig

##### Plotting

In [ ]:
fig_map = barplot_animation(Ps, qs)
fig_map.show()

### Policy trajectory

The trajectory of the MDP for a given policy $\pi$, for each time intant $t$ (assuming that $T$ is the length of the trajectory) is a function that outputs the trajectories of:

- the **state** $S(t)$;
- the **action** $A(t)$; and
- the resulting cumulative **cost** $C(S(t),A(t))$.



> Note: the policy for *finite* horizon MDP with two actions is a mapping from $S\times T$ to $(0,1)$. Therefore, our policy depends on the state and the time as inputs.

#### Function implementation

In [ ]:
def trajectory(pi, q, gamma, K, T, seed=0):
    np.random.seed(seed)
    s = 0
    state_traj = [s]
    action_traj = []
    cost_traj = []
    for t in range(T):
        a = pi[s,t]
        c = cost(s,a,gamma)
        s = simulate_next_state(s,a,q,K)
        state_traj.append(s)
        action_traj.append(a)
        cost_traj.append(c)
    return state_traj, action_traj, cost_traj

#### Visualizing the trajectory

##### Plot function

In [ ]:
def plot_trajectory(states_list, actions_list, costs_list, policy='', color_palette='T10'):
  color_palette = getattr(plotly.colors.qualitative, color_palette)
  fig = make_subplots(rows=3, cols=1, subplot_titles=['Trajectory of States', 'Trajectory of Cumulative Cost', 'Trajectory of Actions'])
  for i, states in enumerate(states_list):
    fig.add_trace(go.Scatter(x=list(range(len(states))), y=states, mode='lines', marker=dict(color=color_palette[i]), name=f'Seed {i}', legendgroup=f'Seed {i}', showlegend=True), row=1, col=1)
    fig.add_trace(go.Scatter(x=list(range(len(costs_list[i]))), y=costs_list[i], mode='lines', marker=dict(color=color_palette[i]), name=f'Seed {i}', legendgroup=f'Seed {i}', showlegend=False), row=2, col=1)
  heatmap_xs, heatmap_ys, heatmap_zs = [], [], []
  for seed in range(len(actions_list)):
    for t in range(len(actions_list[i])):
      heatmap_xs.append(t)
      heatmap_ys.append(seed)
      heatmap_zs.append(actions_list[seed][t])
  fig.add_trace(go.Heatmap(z=heatmap_zs, x=heatmap_xs, y=heatmap_ys, name='Actions', showscale=False, colorscale='Greens', zmin=0, zmax=1, xgap=1, ygap=1), row=3, col=1)
  fig.update_xaxes(title_text='Time instant t', row=3, col=1)
  fig.update_yaxes(title_text='State', row=1, col=1)
  fig.update_yaxes(title_text='Cost', row=2, col=1)
  fig.update_yaxes(title_text='Seed', row=3, col=1)
  avg_cumulative_cost = round(sum([costs[-1] for costs in costs_list]) / len(costs_list), 2)
  fig.update_layout(title=f'Trajectory for Policy "{policy}" with an average cumulative cost of {avg_cumulative_cost}', height=700, showlegend=True)
  return fig

##### Test policy from different seeds

In [ ]:
def test_policy_different_seeds(pi, q, gamma, K, T, n_seeds=10):
  states_list, actions_list, costs_list = [], [], []
  for seed in range(n_seeds):
    states, actions, costs = trajectory(pi, q, gamma, K, T, seed)
    cumulative_costs = [sum(costs[:t]) for t in range(1, len(costs)+1)]
    states_list.append(states)
    actions_list.append(actions)
    costs_list.append(cumulative_costs)
  return states_list, actions_list, costs_list

##### Run simulations for policy

In [ ]:
pi = np.ones((K+1,T))
states_list, actions_list, costs_list = test_policy_different_seeds(pi, q, gamma, K, T)

##### Viewing the plot

In [ ]:
fig_trj = plot_trajectory(states_list, actions_list, costs_list, policy='Always accept')
fig_trj.show()

----
----

# Linear Programming
> Objective: Solving the Markov Chain though Linear Programming.

## Formulating the LP

> The classical LP reformulation of the MDP problem.

$$
\begin{aligned}
& \min_{y \geq 0} \sum_{t=0}^{T-1} \sum_{s,a} C_s^a y_{a,s}(t) \\
& \text{subject to:}  \\
& \quad \quad (a) \quad \; y_{s,0}(t) + y_{s,1}(t) = m_s(t) \quad \quad \forall t \in [0, T-1], \quad \forall s \\
& \quad \quad (b) \quad \; m_s(t) = \sum_{s',a} y_{s',a}(t-1) p_{s',s}^a \quad \forall t \in [1, T-1], \quad \forall s \\
& \quad \quad (c) \quad \; m_s(0) = m_s^0 \quad \quad \forall s \\
\end{aligned}
$$

where:
- $C_s^a$ is the intantaneous cost of taking action $a$ at state $s$.
- (a) Guarantees that the mass of state $s$ at instant $t$ be splitted between $y_{s,0}(t)$ and $y_{s,1}(t)$.
- (b) Guarantees that the mass of state $s$ at instant $t$ be respecting the transitioning probabilities of going from previous state $s'$ at instante $t-1$ to current state $s$.
- (c) Guarantees the initial system state.

## Solving the LP

### CVXPY

In [ ]:
import cvxpy as cp

def solve_lp(K=3, T=5, q=.2, gamma=1, empty_init=True, geq_than=0, sum_up_to_one=True, budget=None):
  # Variables
  m  = cp.Variable((K+1, T), name='mass')
  y0 = cp.Variable((K+1, T), name='y0')  # y0[s,t] = P(s,t) that the action is 0
  y1 = cp.Variable((K+1, T), name='y1')  # y1[s,t] = P(s,t) that the action is 1
  constr = []

  # (a) y_{s,0}(t) + y_{s,1}(t) = m_{s}(t)  # normalization
  for t in range(T):
    for s in range(K+1):
      constr += [y0[s,t] + y1[s,t] == m[s,t]]

  # (b) m_{s}(t) = \sum_{s',a} y_{s',a}(t-1) p_{s',s}^a  # transition probability
  for t in range(1, T):
    for s in range(K+1):
      z = 0
      for s_ in range(K+1):
        p_a0 = inst_MDP(s,s_,0,q,K)
        p_a1 = inst_MDP(s,s_,1,q,K)
        z += p_a0 * y0[s_,t-1] + p_a1 * y1[s_,t-1]
      constr += [m[s,t] == z]

  # (c) m_{s}(0) = m^{0}_s  # initial distribution
  if empty_init:  # all mass in state 0
    constr += [m[0,0] == 1]
    for s in range(1, K+1):
      constr += [m[s,0] == 0]
  else:  # uniform distribution
    for s in range(K+1):
      constr += [m[s,0] == 1/(K+1)]

  # non-negative constraints and y0 + y1 \leq 1
  for t in range(T):
    for s in range(K+1):
      # y \geq 0
      constr += [y0[s,t] >= geq_than]
      constr += [y1[s,t] >= geq_than]
      # y0 + y1 \leq 1
      if sum_up_to_one:
        constr += [y0[s,t] + y1[s,t] <= 1]

  # Budget constraints
  if budget is not None:
    constr += [sum(y1[:,t]) <= budget]

  # Cost function
  Cost = 0
  for t in range(T):
    for s in range(K+1):
      storage_cost = cp.power(s,2)
      cost_action0 = (storage_cost - gamma * y1[s,t])
      cost_action1 = (storage_cost - gamma * y0[s,t])
      Cost += cost_action0 + cost_action1

  problem = cp.Problem(cp.Minimize(Cost), constr)
  result = problem.solve(verbose=False)
  return {
    'result': result,
    'm': m.value,
    'y0': y0.value,
    'y1': y1.value}

### Solving the LP

In [ ]:
result_opt = solve_lp(K, T, q, gamma)

### Obtain the desired policy from the LP

Let $y^*$ be a solution of your LP. Define, for all $0\leq t\leq T-1$, for all $s \in S$ and for all $a \in A$,

$$
\pi_t(a|s)=\left\{\begin{array}{ll}
y_s^a(t)/x_s(t),&\text{if }x_s(t)>0,\\
0&\text{otherwise.}
\end{array}\right.
$$

In [ ]:
def get_policy(y0, y1, bool=True):
  pi = np.zeros((K+1,T))
  for t in range(T):
    for s in range(K+1):
      y0_s = y0[s,t] if y0[s,t] > 0 else 0
      y1_s = y1[s,t] if y1[s,t] > 0 else 0
      m_s = y0_s + y1_s
      if bool:
        pi[s,t] = 1 if y1_s > y0_s else 0  # chooses the action with the highest probability
      else:
        pi[s,t] = 0 if m_s == 0 else round(y1_s / (y0_s + y1_s), 2)
  return pi

## Visualizing the optimal policy

### Getting the Optimal Policy

In [ ]:
pi_opt = get_policy(result_opt['y0'], result_opt['y1'])
states_list_opt, actions_list_opt, costs_list_opt = test_policy_different_seeds(pi_opt, q, gamma, K, T)

### Viewing the Optimal Policy

In [ ]:
fig_opt = plot_trajectory(states_list_opt, actions_list_opt, costs_list_opt, policy='Optimal')
fig_opt.show()

#### Comparison with the "Always Accept" Policy

In [ ]:
fig_trj.show()



----------------------------

----------------------------

# Restless bandit

> Objective: Extend the one queue system for an N-arms restless bandit.

## Solving the problem

In this scenario, instead of just a single queue, we have N queues, where they are attached by a global budget that limit the number of queues that can accept, therefore, to a queue known its acceptance action it depends to known how many queues are also accepting.

> Note: the possible number of combinations of N queues accepting or not is exponential in N, making the problem really intractable.

### Linear program with budget constraint

We can solve the N-arms problem considering just one queue as before as the representative queue of the others. Therefore we can formulate the LP as the one queue system but with a new constraint regarding the budget constraints that the queues shall respect.

#### Formulation of the LP

To implement the projection policy, first we have to solve the following Linear Program:

$$
\begin{aligned}
& \min_{y \geq 0} \sum_{t=0}^{T-1} \sum_{s,a} C_s^a y_{a,s}(t) \\
& \text{subject to:}  \\
& \quad \quad (a) \quad \; y_{s,0}(t) + y_{s,1}(t) = m_s(t) \quad \quad \forall t \in [0, T-1], \quad \forall s \\
& \quad \quad (b) \quad \; m_s(t) = \sum_{s',a} y_{s',a}(t-1) p_{s',s}^a \quad \forall t \in [1, T-1], \quad \forall s \\
& \quad \quad (c) \quad \; m_s(0) = m_s^0 \quad \quad \forall s \\
& \quad \quad (d) \quad \; \sum_s y_{s,1}(t) \leq \alpha, \quad \forall t \in [0, T-1]
\end{aligned}
$$

> Note: It's very similar to the previous LP, but here we have a budget constraint (d) w.r.t $\alpha$.

### Solving with CVXPY

Let's solve this LP problem with the additional budget constraint.

> Note: we can leverage the same function we defined before for solving the LP, but now we just need to change some parameters

In [ ]:
result_budget = solve_lp(K, T, q, gamma, empty_init=False, geq_than=1e-6, sum_up_to_one=False, budget=budget)

Parameters used:
- `empty_init=False`: Previously we initiate the queue empty, now we initiate with a uniform distribution mass between states.
- `geq_than=1e-6`:  Previously we defined a non-negativity constraint where `y` should be equal or greater than $0$, now it's $1e-6$
- `sum_up_to_one=False`: Previously we defined a constraint where `y` should not be greater than $1$, now we remove this constraint.
- `budget=.5`: This is our $\alpha$ budget parameter constraint.

### Getting the policy

Let's create a variable to store our policy $y_s^a(t)$.

In [ ]:
pi_budget = np.zeros((K+1,T,2))
pi_budget[:,:,0] = result_budget['y0']
pi_budget[:,:,1] = result_budget['y1']

## Projection operator

The projection operator converts the LP problem back to the original N-arms problem.

### Formulation of the Quadratic Programming

The formulation is given by:

$$
\begin{aligned}
& \min_{z \geq 0} \sum_{s,a} (z_{s,a} - y_{s,a})^2 \\
& \text{subject to:}  \\
& \quad \quad (a) \quad \; \sum_a z_{s,a} = M_s \quad \quad \forall s \\
& \quad \quad (b) \quad \; \sum_s z_{s,1} \leq \alpha
\end{aligned}
$$

where:
- the squared difference $(z_{s,a} - y_{s,a})^2$ guarantess to $z$ approvimates $y$.
- (a) Guarantees that fractions of arms/queues accepting and rejecting is correct
- (b) Guarantees that the budget is being respected

### Implementation

> Note: we again uses CVXPY to solve this quadratic programming.

In [ ]:
def projec(M,y,alpha,K):
  # Variables
  z = cp.Variable((K+1, 2))
  constr = []

  # (a) \sum_a z_{s,a} = M_s \forall s
  for s in range(K+1):
    constr += [z[s,0] + z[s,1] == M[s]]

  # (b) \sum_s z_{s,1} \leq \alpha.
  for s in range(K+1):
    constr += [sum(z[:,1]) <= alpha]

  # z \geq 0
  for s in range(K+1):
    for a in range(2):
      constr += [z[s,a] >= 0]

  # Cost function
  Cost = 0
  for s in range(K+1):
    for a in range(2):
      Cost += cp.power((z[s,a] - y[s,a]), 2)

  problem = cp.Problem(cp.Minimize(Cost), constr)
  result = problem.solve(verbose=False)
  return(z.value)

## Projection algorithm

At each iteration, the algorithm projects and updates the solution.

In [ ]:
def get_M_hat(alpha, N=10):
  # Variables
  y_round = np.zeros((K+1,2))
  M = np.ones((K+1))
  M = M/(K+1)
  M_hat = np.zeros((K+1,T))
  M_hat[:,0] = M
  for t in range(T-1):
    # Projection step
    y = projec(M_hat[:,t], pi_budget[:,t,:], alpha, K)
    # rounding step
    y_round[:,1] = np.floor(N * y[:,1]) / N
    y_round[:,0] = M - y_round[:,1]
    # Simulation
    for k in range(K+1):
      up_to_for_a1 = (1/N) * np.floor(N*y_round[k,1])
      up_to_for_a0 = M_hat[k,t] - up_to_for_a1
      for s in range(int(N*up_to_for_a1)):
        s_ = simulate_next_state(s,1,q,K)
        M_hat[s_,t+1] += 1/N
      for s in range(int(N*up_to_for_a0)):
        s_ = simulate_next_state(s,0,q,K)
        M_hat[s_,t+1] += 1/N
  return M_hat

## Asymptotically Optimal

When the number of arms $N$ goes to infinity, the optimal policy for the representative queue approximates the N-arms solution.

In [ ]:
N_values = [10 * i for i in range(1, 11)]
m_sol = np.round(result_budget['m'], 1)
norm_values = []
for N in N_values:
  M_hat = get_M_hat(budget, N)
  M_sol = np.round(M_hat, 1)
  norm = np.linalg.norm(M_sol - m_sol)
  norm_values.append(norm)

> Note: we are using the Frobenius norm of the differences of the matrix to express the divergence between the two solutions for each value of $N$.

### Plotting the norm of the difference

Let's compare the norm of the difference between the two mass matrix $\hat{M}$ and $m$ as the number of queues $N$ increases

In [ ]:
fig = go.Figure(data=[go.Bar(x=N_values, y=norm_values)])

# Set the labels and title
fig.update_layout(
  xaxis_title='Number of Arms (N)',
  yaxis_title='Norm Value',
  title='Decaying Norm as N Increases'
)

# Show the plot
fig.show()